<a href="https://colab.research.google.com/github/rawanamrrr/DeepLearning1/blob/main/GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch torchvision torchaudio
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv, GCNConv
import torch_geometric.transforms as T

In [5]:
dataset = Planetoid(root='data/Cora', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0]

Processing...
Done!


In [6]:
class GATNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=8, dropout=0.6):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [7]:
class GCNNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [18]:
def train(model, data, optimizer, criterion, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                _, pred = model(data.x, data.edge_index).max(dim=1)
                correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
                val_acc = correct.item() / data.val_mask.sum().item()
                print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}')
            model.train()

In [19]:
def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = (pred[mask] == data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
    return acc

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

In [21]:
gat_model = GATNet(dataset.num_features, 8, dataset.num_classes).to(device)
gat_optimizer = torch.optim.Adam(gat_model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

In [22]:
print("Training GAT Model:")
train(gat_model, data, gat_optimizer, criterion, epochs=200)

Training GAT Model:
Epoch: 000, Loss: 1.9493, Val Acc: 0.2480
Epoch: 010, Loss: 1.8766, Val Acc: 0.7680
Epoch: 020, Loss: 1.7797, Val Acc: 0.7500
Epoch: 030, Loss: 1.6653, Val Acc: 0.8000
Epoch: 040, Loss: 1.5640, Val Acc: 0.8180
Epoch: 050, Loss: 1.4069, Val Acc: 0.8000
Epoch: 060, Loss: 1.2705, Val Acc: 0.8020
Epoch: 070, Loss: 1.1294, Val Acc: 0.7800
Epoch: 080, Loss: 1.0217, Val Acc: 0.8040
Epoch: 090, Loss: 1.1074, Val Acc: 0.7980
Epoch: 100, Loss: 0.8920, Val Acc: 0.7980
Epoch: 110, Loss: 0.8886, Val Acc: 0.8120
Epoch: 120, Loss: 0.8271, Val Acc: 0.7940
Epoch: 130, Loss: 0.7911, Val Acc: 0.8060
Epoch: 140, Loss: 0.7415, Val Acc: 0.8000
Epoch: 150, Loss: 0.7597, Val Acc: 0.7940
Epoch: 160, Loss: 0.7760, Val Acc: 0.7980
Epoch: 170, Loss: 0.7447, Val Acc: 0.8040
Epoch: 180, Loss: 0.6787, Val Acc: 0.8000
Epoch: 190, Loss: 0.6857, Val Acc: 0.8020


In [23]:
gat_test_acc = evaluate(gat_model, data, data.test_mask)
print(f'GAT Test Accuracy: {gat_test_acc:.4f}')

GAT Test Accuracy: 0.8080


In [24]:
gcn_model = GCNNet(dataset.num_features, 16, dataset.num_classes).to(device)
gcn_optimizer = torch.optim.Adam(gcn_model.parameters(), lr=0.01, weight_decay=5e-4)

In [25]:
print("\nTraining GCN Model:")
train(gcn_model, data, gcn_optimizer, criterion, epochs=200)


Training GCN Model:
Epoch: 000, Loss: 1.9460, Val Acc: 0.2520
Epoch: 010, Loss: 1.8514, Val Acc: 0.6640
Epoch: 020, Loss: 1.7081, Val Acc: 0.7520
Epoch: 030, Loss: 1.5193, Val Acc: 0.7500
Epoch: 040, Loss: 1.2690, Val Acc: 0.7640
Epoch: 050, Loss: 1.1155, Val Acc: 0.7700
Epoch: 060, Loss: 0.9411, Val Acc: 0.8000
Epoch: 070, Loss: 0.7609, Val Acc: 0.8000
Epoch: 080, Loss: 0.7084, Val Acc: 0.7940
Epoch: 090, Loss: 0.6302, Val Acc: 0.8020
Epoch: 100, Loss: 0.5303, Val Acc: 0.7940
Epoch: 110, Loss: 0.5034, Val Acc: 0.7980
Epoch: 120, Loss: 0.4833, Val Acc: 0.7960
Epoch: 130, Loss: 0.4561, Val Acc: 0.7960
Epoch: 140, Loss: 0.4341, Val Acc: 0.7960
Epoch: 150, Loss: 0.3984, Val Acc: 0.7980
Epoch: 160, Loss: 0.4204, Val Acc: 0.7940
Epoch: 170, Loss: 0.3806, Val Acc: 0.7960
Epoch: 180, Loss: 0.3796, Val Acc: 0.7900
Epoch: 190, Loss: 0.3477, Val Acc: 0.7920


In [26]:
gcn_test_acc = evaluate(gcn_model, data, data.test_mask)
print(f'GCN Test Accuracy: {gcn_test_acc:.4f}')

GCN Test Accuracy: 0.8170
